# 爬取 百合网(http://www.baihe.com/home.shtml) 的用户数据

## 百合网站的模块
### 1. 用户成功故事(http://story.baihe.com/ ) - 用户分享的恋爱故事列表
<img style="float: middle;"  src="故事列表.png"  width="70%">
### 2. 用户个人空间(http://story.baihe.com/space.php?id=uid) 
###    每一位用户都有一个与个人id相关联的个人空间， 记载了该用户的个人恋爱故事
<img style="float: middle;"  src="个人空间.png"  width="80%">
### 3. 用户基础数据(http://profile1.baihe.com/?oppID=uid) - 每一位用户都有一个与个人id个人展示页面
<img style="float: middle;"  src="个人信息.png"  width="80%">

In [6]:
import re
import requests
import chardet
from bs4 import BeautifulSoup
import pickle

def request_html(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    html = requests.get(url, headers=headers)
    charset = chardet.detect(html.content)['encoding']
    html = html.content.decode(charset)
    return html

# 爬取用户故事列表

In [16]:
base_url = 'http://story.baihe.com/story.php?story_list&p=%d'
  
story_list = dict()
for i in range(1, 292):
    html = request_html(base_url%i)
    soup = BeautifulSoup(html,'html.parser')
    for dd in soup.find('div',{'class':'storyMore'}).find_all('dd'):
        try:
            a = dd.div.find_all('a')
            user_id = a[0].get('href').split('=')[-1]
            story_id = a[1].get('href').split('=')[-1]
        except:
            print(i)
            continue
        state = dd.find('span',{'class':'state'}).text[5:]
        story_list[story_id] = {'state': state, 'user': user_id}
        
print(story_list.pop('27274'))
print('所有故事状态：')
print({s['state'] for s in story_list.values()})

{'state': '宣布结婚了', 'user': '87871348'}
所有故事状态：
{'', '宣布结婚了', '甜蜜恋爱了', '还在寻觅中', '开始约会了'}


In [18]:
num_of_user = len({s['user'] for s in story_list.values()})
num_of_story = len(story_list)
if num_of_user == num_of_story:
    print('每个用户只有一个故事出现在列表中')
else:
    print('每个用户有多个故事出现在列表中')

每个用户有多个故事出现在列表中


In [42]:
user2state = dict()
story_id = sorted(story_list.keys(), key=lambda x: int(x))
for story in story_id:
    user = story_list[story]['user']
    if not story_list[story]['state']:
        continue
    if user not in user2state:
        user2state[user] = story_list[story]['state']
    else:
        if story_list[story]['state'] == user2state[user].split('->')[-1]:
            continue
        else:
            user2state[user] += '->' + story_list[story]['state']

from collections import Counter
counter = Counter(user2state.values())
items = sorted(counter.items(), key=lambda x:x[1], reverse=True)
for key, value in items:
    print('%3d个用户\t%s'%(value, key))

816个用户	宣布结婚了
660个用户	甜蜜恋爱了
 78个用户	开始约会了
 39个用户	甜蜜恋爱了->宣布结婚了
 23个用户	还在寻觅中
 13个用户	宣布结婚了->甜蜜恋爱了
  6个用户	开始约会了->甜蜜恋爱了
  4个用户	开始约会了->宣布结婚了
  3个用户	宣布结婚了->甜蜜恋爱了->宣布结婚了
  2个用户	还在寻觅中->甜蜜恋爱了
  2个用户	宣布结婚了->还在寻觅中
  1个用户	还在寻觅中->甜蜜恋爱了->宣布结婚了
  1个用户	甜蜜恋爱了->宣布结婚了->甜蜜恋爱了->宣布结婚了
  1个用户	甜蜜恋爱了->还在寻觅中
  1个用户	宣布结婚了->甜蜜恋爱了->宣布结婚了->甜蜜恋爱了->宣布结婚了->甜蜜恋爱了
  1个用户	甜蜜恋爱了->开始约会了


In [43]:
pickle.dump(story_list, open('story_list.pkl', 'wb'))

# 爬取用户个人空间

In [232]:
# 提取情侣id信息
def user_infor(soup):
    name = soup.p.text
    if 'profile1' not in soup.a.get('href'):
        return {}
    uid = soup.a.get('href').split('=')[-1]
    if uid == '-1':
        return {}
    try:
        age = re.findall('(\d+)',soup.a.text)[0]
        area = soup.a.text.split()[-1]
        img = soup.img.get('src')
        return {'name':name, 'img':img, 'age':age, 'area':area, 'uid':uid}
    except:
        return {'uid':uid}

# 提取个人空间页面信息
def extract_kongjian(html):
    data = dict()
    soup = BeautifulSoup(html,'lxml')
    
    personal = soup.find('div',{'class':'personal'})
    data['state'] = personal.find('div',{'class':'state'}).text
    data['duration'] = re.sub('上传我的故事','',personal.find_all('li')[-1].text)
    
    user = personal.find_all('li')[:2]
    data['user1'] = user_infor(user[0])
    data['user2'] = user_infor(user[1])
    
    data['article'] = []
    while True:
        for div in soup.find_all('div',{'class':'article'}):
            article = dict()
            article['title'] = div.div.a.text
            article['story_id'] = div.div.a.get('href').split('=')[-1]
            article['time'] = div.div.span.text
            article['content'] = re.sub('\s','',div.find('div',{'class':'cont'}).text)
            data['article'].append(article)
        page_bar = soup.find('div',{'class':'page_now'})
        if not page_bar:
            break
        else:
            now = page_bar.find('span')
            next_page = now.find_next('a')
            if next_page.text == '后页':
                break
            else:
                html = requests.get('http://story.baihe.com/'+next_page.get('href')).text
                soup = BeautifulSoup(html,'lxml')
    return data

In [168]:
stroy_list = pickle.load(open('story_list.pkl', 'rb'))
users = {s['user'] for s in story_list.values()}
users = [u for u in users if u != 'space.php']
# kongjian = dict()
base_url = 'http://story.baihe.com/space.php?id='
for i, user in enumerate(users):
    if user in ['53702191', '56648612']:
        continue
    if user in kongjian:
        continue
    html = requests.get(base_url + user).text
    kongjian[user] = extract_kongjian(html)
    if not kongjian[user]['user1'] or kongjian[user]['user1']['uid'] != user:
        raise Exception
        
print(kongjian.pop('50122651'))

{'state': '宣布结婚', 'duration': '211天后恋爱', 'user1': {'name': '云艾', 'img': 'http://photo12.baihe.com/2007/01/12/100_100/5FCEE5867FA1B6CCEA5E325648778799.jpg', 'age': '38', 'area': '福建福州', 'uid': '50122651'}, 'user2': {}, 'article': [{'title': '我们就要这样恋爱一辈子!', 'story_id': '547', 'time': '2007-08-03 15:05:17', 'content': '(1)2007年3月15日3/15消费者权益日，怎么会如此快就载入我的大脑CPU，那天太阳很大，气温有点高，我的脑袋有轻微的晕眩，犹豫了几分种，思索着该以什么样的形象、着装风格出现在你面前，突然觉得没有必要，就以一颗平常心吧，说不准发生什么呢，感觉是一种奇怪的东西。我宁愿相信自己的第6感，我想我们对彼此的印象应该都不错！周四过后，周日晚在温泉百合上岛咖啡厅，我们又见面...'}]}


In [170]:
# 将所有人的伴侣加入待爬取列表
users = {s['user'] for s in story_list.values()}
users.remove('space.php')
users.update({k['user2']['uid'] for k in kongjian.values() if k and 'uid' in k['user2']})
for user in users:
    if user in kongjian:
        continue
    html = requests.get(base_url + user).text
    if len(html) == 20324:
        kongjian[user] = {}
        continue
    else:
        kongjian[user] = extract_kongjian(html)

In [238]:
pickle.dump(kongjian, open('kongjian2.pkl', 'wb'))

In [3]:
kongjian = pickle.load(open('kongjian2.pkl', 'rb'))

In [4]:
wrong = []
for user, value in kongjian.items():
    if len(value) == 0:
        wrong.append(user)
        continue
    if 'uid' in value['user2'] and value['user2']['uid'] == user:
        wrong.append(user)
for w in wrong:
    kongjian.pop(w)

In [5]:
lovers = dict()
for user, value in kongjian.items():
    if len(value) == 0:
        continue
    if len(value['user2']) == 0:
        continue
    lovers[user] = value['user2']['uid']
            
# 提取互为情侣的数据
pairs = set()
for user in lovers:
    # 若A的伴侣是B，则B的伴侣应为A
    if lovers[user] in lovers and lovers[lovers[user]] != user:
        print('%s的情侣是%s, 但是%s的情侣是%s'%(user, lovers[user], lovers[user], lovers[lovers[user]]))
    else:
        pairs.add((min(user,lovers[user]),max(user,lovers[user])))


58477845的情侣是71871256, 但是71871256的情侣是65190937
50451794的情侣是1196009, 但是1196009的情侣是719161
66271212的情侣是67311290, 但是67311290的情侣是66410644
50701719的情侣是50642151, 但是50642151的情侣是3349658


## 爬取所有的用户故事， 可以看出用户的恋爱状态是否发生变化了

In [329]:
def extract_story(html):
    data = dict()
    soup = BeautifulSoup(html,'html.parser')
    data['title'] = soup.h3.text
    data['time'] = ' '.join(soup.h4.text.split()[:2])
    data['article'] = soup.find('div',{'class':'details'}).div
    personal = soup.find('div',{'class':'personal'})
    data['state'] = personal.find('div',{'class':'state'}).text
    data['duration'] = re.sub('上传我的故事','',personal.find_all('li')[-1].text)
    
    user = personal.find_all('li')[:2]
    data['user1'] = user_infor(user[0])
    data['user2'] = user_infor(user[1])
    return data

In [ ]:
import os
# stories = dict()
for d in os.listdir('html2'):
    if d in stories:
        continue
    with open('html2/'+d, 'r', encoding='utf8') as f:
        html = f.read()
    try:
        stories[d] = extract_story(html)
    except:
        continue

In [339]:
len(stories)

12110

In [340]:
{v['state'] for v in stories.values()}

{'宣布结婚'}

In [ ]:
base_url = 'http://story.baihe.com/story.php?id='
stories = dict()
for i in range(1000, 27269):
    html = base_url + str(i)
    stories[str(i)] = extract_story(html)


In [241]:
stories = {article['story_id'] for value in kongjian.values() for article in value['article']}

In [327]:
stories = {int(i) for i in stories}
print(max(stories))
print(min(stories))

27269
17


In [328]:
(27269 - 17) - len(stories)

25001

# 提取用户信息 —— 需要登陆（通过设置cookies）

In [272]:
from extract_userinfo import *
import json
import re

# 设置cookie
from requests.cookies import RequestsCookieJar
url = "http://profile1.baihe.com/?showType=2012&oppID=78057814"
 
cookie_jar = RequestsCookieJar()
with open('cookies.txt','r') as f:
    for line in f.readlines():
        key, value = line.split()[:2]
        cookie_jar.set(key, value)

authen_url = 'http://usericon.baihe.com/index/getUserAttestation?jsonCallBack=jQuery183016901437235994476_1532274807369&userID='
def authen_info(uid):
    text = requests.get(authen_url+uid).text
    text = re.findall('\((.*?)\)',text)[0]
    text = re.sub('null','0',text)
    return json.loads(text)['data']

In [307]:
blank = ['56168247', '76714707', '56332940', '1400', '56656421', '56570273']
base_url = 'http://profile1.baihe.com/?oppID='
users = {u for p in pairs for u in p}
# woman = dict()
# man = dict()
for i, user in enumerate(users):
    if user in man or user in woman:
        continue
    if i % 100 == 0:
        print(i)
    if user in blank:
        continue
    data = authen_info(user)

    url = base_url + user
    html = requests.get(url, cookies=cookie_jar).text
    if len(html) < 2500:
        continue
    soup = BeautifulSoup(html,'html.parser')
    with open('users/%s' % user, 'w', encoding='utf8') as f:
        f.write(str(soup.find('div',{'id':'BAIHE'})))

    if data['genderChn'] == '她':
        data.update(extract_woman(soup))
        woman[data['userID']] = data
    else:
        data.update(extract_man(soup))
        man[data['userID']] = data

1400
1600


In [296]:
pickle.dump(man, open('man.pkl', 'wb'))
pickle.dump(woman, open('woman.pkl', 'wb'))

In [1]:
import pickle
woman = pickle.load(open('woman.pkl', 'rb'))
man = pickle.load(open('man.pkl', 'rb'))

In [6]:
wrong = []
for user1, user2 in pairs:
    flag1 = user1 in man and user2 in woman
    flag2 = user1 in woman and user2 in man
    if flag1 or flag2:
        continue
    wrong.append((user1,user2))
pairs = pairs - set(wrong)

In [8]:
keys = set()
for v in woman.values():
    keys = keys.union(v.keys())
for v in man.values():
    keys = keys.union(v.keys())
keys.remove('userID')
keys = ['userID'] + sorted(keys)
print(keys)

import csv
def write_csv(file, data, keys):
    with open(file, 'w', encoding='utf8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(keys)
        for value in data.values():
            d = [value[key] if key in value else '' for key in keys]
            writer.writerow(d)
            
# write_csv('man.csv', man, keys)
write_csv('woman.csv', woman, keys)

['userID', 'age', 'area', 'creditedNum', 'education', 'genderChn', 'height', 'intro', 'isCreditedByEducation', 'isCreditedById5', 'isCreditedByMobile', 'isCreditedBySesame', 'isCreditedBySfz', 'name', 'nickName', 'picture', 'status', 'tags', '住房', '体型', '体重', '偏爱约会方式', '公司性质', '公司行业', '厨艺状况', '婚姻', '婚姻状况', '学历', '宗教信仰', '家中排行', '家乡', '家务分工', '属相', '工作状况', '希望对方看重', '年龄', '恋爱类型', '想何时结婚', '户口', '所在地区', '所学专业', '择偶要求_有无子女', '掌握语言', '星座', '是否吸烟', '是否喝酒', '是否想要小孩', '月收入', '月薪', '有无子女', '期待婚礼形式', '母亲工作', '毕业学校', '民族', '父亲工作', '父母医保', '父母情况', '父母经济', '生活作息', '相貌自评', '职业', '血型', '购房', '购房情况', '购车', '身高']


In [346]:
mkeys = set()
for v in man.values():
    mkeys = mkeys.union(v.keys())
print(mkeys)

{'isCreditedById5', '身高', '购房情况', '有无子女', '属相', '恋爱类型', '月收入', '家乡', '是否吸烟', '住房', '星座', 'creditedNum', '父母医保', '婚姻', '体型', '希望对方看重', '父母情况', '厨艺状况', 'isCreditedBySesame', '公司行业', 'isCreditedBySfz', '相貌自评', 'education', '户口', '父亲工作', '婚姻状况', '父母经济', '宗教信仰', 'nickName', '期待婚礼形式', 'genderChn', '是否喝酒', '家务分工', '家中排行', '工作状况', '学历', '年龄', 'isCreditedByMobile', '是否想要小孩', 'intro', 'age', 'picture', '所在地区', '职业', 'isCreditedByEducation', '掌握语言', 'height', '择偶要求_有无子女', '偏爱约会方式', 'status', '月薪', '购车', '想何时结婚', '生活作息', '公司性质', 'userID', 'tags', '毕业学校', 'name', 'area', '民族', '所学专业', '血型', '母亲工作', '体重'}


In [350]:
print(keys.intersection(mkeys))

{'想何时结婚', '生活作息', '公司性质', 'isCreditedById5', '公司行业', 'isCreditedBySfz', '身高', 'userID', '相貌自评', '购房情况', 'tags', '学历', '年龄', 'education', '有无子女', '户口', 'isCreditedByMobile', '是否想要小孩', 'intro', '父亲工作', '属相', '恋爱类型', '月收入', 'age', 'picture', '所在地区', '毕业学校', '家乡', '是否吸烟', '职业', '婚姻状况', 'isCreditedByEducation', '父母经济', '星座', '掌握语言', '宗教信仰', 'height', '择偶要求_有无子女', 'creditedNum', '父母医保', 'nickName', 'name', '期待婚礼形式', '体型', 'genderChn', '偏爱约会方式', '是否喝酒', 'area', '民族', '希望对方看重', '所学专业', 'status', '血型', '家务分工', '母亲工作', '月薪', '父母情况', '厨艺状况', '购车', '体重', '家中排行', '工作状况', 'isCreditedBySesame'}


In [348]:
print(keys-mkeys)

{'购房'}


In [349]:
print(mkeys-keys)

{'住房', '婚姻'}


In [275]:
def extract_man(soup):
    data = dict()
    data['name'] = soup.find('div',{'class':'name'}).text.split()[0]

    a,b = soup.find('div',{'class':'manPic'}).dl.find_all('dt')[-1].text.split('：')
    data[a] = b
    
    divs = soup.find_all('div',{'class':'inter'})
    label = soup.find('div',{'class':'inter label'})
    if label:
        divs.pop(divs.index(label))
        #data['sex'] = 
        data['tags'] = '/'.join([i.text for i in label.find_all('span')])
    
    #data['id'] = 
    splits = [i for i in re.findall('>(.*?)<', str(divs[-1])) if i != '/']
    # splits = [i.strip() for i in soup.find('div',{'class':'inter'}).text.split('/')]
    data['age'], data['height'], data['education'], data['area'] = splits[:4]
    
    # 是否有照片
    div = soup.find('div',{'class':'manPic'}).find('div',{'class':'noPic'})
    if div:
        data['picture'] = '无'
    else:
        data['picture'] = '有'
        
    # 恋爱状态
    div = soup.find('div',{'class':'womanPic'})
    if not div:
        div = soup.find('div',{'class':'manPic'})
    if div.em:
        data['status'] = div.em.text
    else:
        data['status'] = ''
    
    # 爬取表格
    for infor in ['data', 'perData']:
        div = soup.find('div',{'class': infor})
        dt = div.find_all('dt')
        dd = [d for d in div.find_all('dd') if not d.get('style')]
        for i in range(len(dt)):
            key = re.sub('\s+','',dt[i].text[:-1])
            value = re.sub('\s+','',dd[i].text.strip())
            if key in data and value != data[key]:
                if key == '有无子女':
                    data['择偶要求_有无子女'] = value
                elif key == '相貌自评':
                    continue
                else:
                    raise Exception('这个键有问题 %s %s %s' % (key, value, data[key]))
            else:
                data[key] = value
    data['intro'] = re.sub('\s+','',soup.find('div',{'class':'intr'}).text.strip())
    if '择偶要求_有无子女' not in data:
        data['择偶要求_有无子女'] = data['有无子女']
    if '登录后可见' in data.values():
        raise Exception('登陆问题')
    return data

In [277]:
def extract_woman(soup):
    data = dict()
    data['name'] = soup.find('div',{'class':'name'}).text.split()[0]
       
    a,b = soup.find('div',{'class':'data'}).find_all('dt')[-1].text.split('：')
    data[a] = b
    
    divs = soup.find_all('div',{'class':'inter'})
    label = soup.find('div',{'class':'inter label'})
    if label:
        divs.pop(divs.index(label))
        #data['sex'] = 
        data['tags'] = '/'.join([i.text for i in label.find_all('span')])
    
    #data['id'] = 
    splits = [i for i in re.findall('>(.*?)<', str(divs[-1])) if i != '/']
    # splits = [i.strip() for i in soup.find('div',{'class':'inter'}).text.split('/')]
    data['age'], data['height'], data['education'], data['area'] = splits[:4]
    
    # 是否有照片
    div = soup.find('div',{'class':'womanPic'}).find('div',{'class':'noPic'})
    if div:
        data['picture'] = '无'
    else:
        data['picture'] = '有'
        
    # 恋爱状态
    div = soup.find('div',{'class':'womanPic'})
    if not div:
        div = soup.find('div',{'class':'manPic'})
    if div.em:
        data['status'] = div.em.text
    else:
        data['status'] = ''
    
    # 爬取表格
    infor = 'perData'
    div = soup.find('div',{'class': infor})
    dt = div.find_all('dt')
    dd = [d for d in div.find_all('dd') if not d.get('style')]
    for i in range(len(dt)):
        key = re.sub('\s+','',dt[i].text[:-1])
        value = re.sub('\s+','',dd[i].text.strip())
        if key in data and value != data[key]:
            if key == '有无子女':
                data['择偶要求_有无子女'] = value
            elif key == '相貌自评':
                continue
            else:
                raise Exception('这个键有问题 %s %s %s' % (key, value, data[key]))
        else:
            data[key] = value
    data['intro'] = re.sub('\s+','',soup.find('div',{'class':'intr'}).text.strip())
    
    if '择偶要求_有无子女' not in data:
        data['择偶要求_有无子女'] = data['有无子女']
    if '登录后可见' in data.values():
        raise Exception('登陆问题')
    return data
